# Представляю вам мою версию проекта предсказания цены автомобиля.

Первым делом подгрузим нужные библиотеки.

In [1]:
import pandas as pd 
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import ExtraTreeRegressor

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from tqdm import tqdm
%matplotlib inline

Зададим константы для настройки модели.

In [2]:
RANDOM_SEED = 42  
VAL_SIZE   = 0.33
n_folds = 5
iterations = 1300
learning_rate = 0.05
VERSION = 6

Далее подгрузим наши данные. Для обучения модели будем использовать https://www.kaggle.com/macsunmood/autoru-parsed-0603-1304 датасет с kaggle.com 

In [3]:
data_train = pd.read_csv('new_data_99_06_03_13_04.csv')
data_train= data_train.drop_duplicates().reset_index(drop = True) #чистим данные от дубликатов и сбрасываем индек.
data_test = pd.read_csv('test.csv') #загружаем тестовый датасет
sample_submission = pd.read_csv('sample_submission.csv') 

Давайте посмотрим на данные в обучающем датасете.

In [4]:
display(data_train.head())
data_train.info()

,Unnamed: 0,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,...,description,mileage,Комплектация,Привод,Руль,Владельцы,ПТС,Таможня,Владение,Price
0,0,Седан,AUDI,040001,бензин,1990.0,2.3 MT (133 л.с.),4.0,1991,MECHANICAL,...,\nБыстрым торг.Обмен интересен на авто с неисп...,10000,{'id': '0'},передний,LEFT,3.0,ORIGINAL,True,NaN,135000
1,1,Седан,AUDI,040001,бензин,1988.0,2.0 MT (115 л.с.),4.0,1989,MECHANICAL,...,На ходу!Очень много сделано!До аварии ездил 1....,300000,{'id': '0'},передний,LEFT,3.0,DUPLICATE,True,NaN,42000
2,2,Универсал 5 дв.,AUDI,CACECB,бензин,1990.0,2.3 MT (133 л.с.),5.0,1991,MECHANICAL,...,Новый перешитый потолок. \nМашина на полном ходу,205636,{'id': '0'},передний,LEFT,2.0,ORIGINAL,True,NaN,200000
3,3,Седан,AUDI,040001,бензин,1990.0,2.3 MT (133 л.с.),4.0,1991,MECHANICAL,...,"Двигатель в порядке не дымит, масло не ест, ко...",450000,{'id': '0'},передний,LEFT,3.0,ORIGINAL,True,NaN,119000
4,4,Седан,AUDI,97948F,бензин,1990.0,2.3 MT (133 л.с.),4.0,1991,MECHANICAL,...,Автомобиль на полном ходу. Кузов и двигатель в...,275250,{'id': '0'},передний,LEFT,3.0,DUPLICATE,True,"{'year': 2013, 'month': 8}",125000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109080 entries, 0 to 109079
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            109080 non-null  int64  
 1   bodyType              109079 non-null  object 
 2   brand                 109080 non-null  object 
 3   color                 109080 non-null  object 
 4   fuelType              109080 non-null  object 
 5   modelDate             109079 non-null  float64
 6   name                  109079 non-null  object 
 7   numberOfDoors         109079 non-null  float64
 8   productionDate        109080 non-null  int64  
 9   vehicleConfiguration  109079 non-null  object 
 10  engineDisplacement    109079 non-null  object 
 11  enginePower           109079 non-null  float64
 12  description           105829 non-null  object 
 13  mileage               109080 non-null  int64  
 14  Комплектация          109080 non-null  object 
 15  

Давайте далее разберём основные колонки с данными.
Первой колонкой идет "Unnamed: 0" как мы видим это колонка с индексом, и она нам не нужна. може смело ее удалять.

In [5]:
data_train.drop(['Unnamed: 0'], axis = 1, inplace = True)

Посмотрим так же на тестовый датасет

In [6]:
display(data_test.head())
data_test.info()

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id
0,седан,BMW,чёрный,дизель,2016.0,520d 2.0d AT (190 л.с.),4.0,2017.0,SEDAN AUTOMATIC 2.0,автоматическая,...,158836.0,"['[{""name"":""Безопасность"",""values"":[""Антипробу...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,0
1,седан,BMW,белый,дизель,2018.0,318d 2.0d AT (150 л.с.),4.0,2019.0,SEDAN AUTOMATIC 2.0,автоматическая,...,10.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,1
2,седан,BMW,синий,бензин,2009.0,550i xDrive 4.4 AT (407 л.с.) 4WD,4.0,2012.0,SEDAN AUTOMATIC 4.4,автоматическая,...,120000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,7 лет и 2 месяца,2
3,внедорожник 5 дв.,BMW,белый,дизель,2014.0,30d 3.0d AT (249 л.с.) 4WD,5.0,2015.0,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,...,111466.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,3
4,внедорожник 5 дв.,BMW,синий,дизель,2014.0,M50d 3.0d AT (381 л.с.) 4WD,5.0,2019.0,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,...,11891.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3837 entries, 0 to 3836
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              3837 non-null   object 
 1   brand                 3837 non-null   object 
 2   color                 3837 non-null   object 
 3   fuelType              3837 non-null   object 
 4   modelDate             3837 non-null   float64
 5   name                  3837 non-null   object 
 6   numberOfDoors         3837 non-null   float64
 7   productionDate        3837 non-null   float64
 8   vehicleConfiguration  3837 non-null   object 
 9   vehicleTransmission   3837 non-null   object 
 10  engineDisplacement    3837 non-null   object 
 11  enginePower           3837 non-null   object 
 12  description           3837 non-null   object 
 13  mileage               3837 non-null   float64
 14  Комплектация          3837 non-null   object 
 15  Привод               

Давайте приведём данные в колонке 'bodyType' обучающего датасета к виду как и в тестовом датасете.

In [7]:
data_train['bodyType'] = data_train['bodyType'].apply(lambda x: str(x))
def bodyType(row):
    for body_type in ['внедорожник 5 дв.', 'седан-хардтоп', 'хэтчбек 5 дв.','внедорожник 3 дв.', 
                      'купе-хардтоп', 'внедорожник открытый','хэтчбек 3 дв.', 'пикап двойная кабина',
                      'пикап полуторная кабина', 'пикап одинарная кабина', 'седан 2 дв.','универсал 5 дв.', 
                      'родстер', 'кабриолет','фургон', 'микровэн','минивэн', 'компактвэн','лифтбек',
                      'купе','тарга', 'седан','лимузин']:
        if row.lower().startswith(body_type):
            return body_type
data_train['bodyType'] = data_train['bodyType'].apply(bodyType)

Так же приведём вид колонки цвета в обучающем датасете к виду из тестового.

In [8]:
data_train['color'] = data_train['color'].map({'CACECB': 'серебристый', 'FAFBFB':'белый', 'EE1D19':'красный', '97948F':'серый', 
                                     '660099':'пурпурный', '040001':'чёрный','4A2197':'фиолетовый', 
                                     '200204':'коричневый','0000CC':'синий', '007F00':'зелёный', 'C49648':'бежевый',
                                     '22A0F8':'голубой','DEA522':'золотистый','FFD600': 'жёлтый', 'FF8649':'оранжевый',
                                     'FFC0CB':'розовый'})

Дальше прводим к общему виду колонки Руль и ПТС

In [9]:
data_train['Руль'] = data_train['Руль'].map({'LEFT': 'Левый', 'RIGHT': 'Правый'})
data_train['ПТС'] = data_train['ПТС'].map({'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат'}).fillna('Оригинал')

Приведём столбец  со сроком жизни автомобиля к виду из тестовой подборке данных.

In [10]:
data_train['Владение'] = data_train['Владение'].fillna('nodata')
def months_to_sent(months):
    if months == 1:
        return f'{months} месяц'
    elif 2 <= months <= 4:
        return f'{months} месяца'
    return f'{months} месяцев'
def years_to_sent(years):
    if 11 <= years <= 14 or 5 <= years%10 <= 9 or years%10 == 0:
        return f'{years} лет'
    elif years%10 == 1:
        return f'{years} год'
    elif 2 <= years%10 <= 4:
        return f'{years} годa'
def tenure(row):
    row = re.findall('\d+',row)
    if row != []:
        years = 2020 - (int(row[0])+1)
        months = 2 +(12 - int(row[1]))
        if years < 0:
            return months_to_sent(int(row[1]))
        elif years == 0 and months < 12:
            return months_to_sent(months)
        elif years >= 0 and months == 12:
            return years_to_sent(years + 1)
        elif years >= 0 and months > 12:
            return years_to_sent(years + 1)+' и '+months_to_sent(months - 12)
        elif years > 0 and months < 12:
            return years_to_sent(years)+' и '+months_to_sent(months)
        return None
data_train['Владение'] = data_train['Владение'].apply(tenure)

Столбец таможны содержит всего одно значение , можем его спокойно удалить.

In [11]:
data_train = data_train.drop(['Таможня'], axis = 1)

Дальше поиграемся с тестовыми данными.

In [12]:
# удалим ненужные столбцы
data_test = data_test.drop(['id','Состояние', 'Таможня', 'vehicleTransmission'], axis = 1)
  # приравняем к единому числовому варианту
for feature in ['modelDate', 'numberOfDoors', 'productionDate', 'mileage']:
    data_test[feature] = data_test[feature].astype('int64')
#vehicleConfiguration, выведем только трансмиссию
data_test['vehicleConfiguration'] = data_test['vehicleConfiguration'].apply(lambda x: x.split()[1])
data_test['engineDisplacement'] = data_test['engineDisplacement'].apply(lambda x: x.split()[0])
#enginePower, оставим только цифру в enginePower
data_test['enginePower'] = data_test['enginePower'].apply(lambda x: int(x.split()[0]))
# Владельцы, оставим только цифру
data_test['Владельцы'] = data_test['Владельцы'].apply(lambda x: int(x.split()[0]))

In [13]:
# train = train_(data_train)
data_train.dropna(axis = 0, thresh=18,inplace=True)#строки , где слишком много пропущенных значений.
# test = test_(data_test)

In [14]:
data_train.Price=data_train.Price.apply(lambda x: np.log(x))

Создаём в каждом датафрейме колонку для идентификации их , а так же дополняем тестовый датофрейм не достающими значениями. И обьединяем их для большего удобства дальнейшей обработки.

In [15]:
data_train['sample'] = 1
data_test['sample'] = 0
data_test['Price'] = 0
data = data_test.append(data_train, sort=False).reset_index(drop=True)

Вычленяем из колонки с описанием отдельные параметры

In [16]:
data['description'] = data['description'].fillna('[]')
data['description_len'] = data['description'].apply(lambda x: len(x.split()))
data['description_word'] = data['description'].apply(lambda x: [str(i).lower() for i in x.split()])

Превращаем описание автомобилей в дамми даные.

In [17]:
data['leather']= data['description_word'].apply(lambda x: 
                                                1 if ('темный' and 'салон') in x else 0)
data['carter']= data['description_word'].apply(lambda x: 
                                                1 if ('защита' and 'картера') in x else 0)
data['ABS']= data['description_word'].apply(lambda x: 
                                                1 if ('антиблокировочная' and 'система') in x else 0)
data['airbags']= data['description_word'].apply(lambda x: 
                                                1 if ('подушки' and 'безопасности') in x else 0)
data['immob']= data['description_word'].apply(lambda x: 
                                                1 if ('иммобилайзер') in x else 0)
data['central_locking']= data['description_word'].apply(lambda x: 
                                                1 if ('центральный' and 'замок') in x else 0)
data['on_board_computer']= data['description_word'].apply(lambda x: 
                                                1 if ('бортовой' and 'компьютер') in x else 0)
data['cruise_control']= data['description_word'].apply(lambda x: 
                                                1 if ('круиз-контроль') in x else 0)
data['climat_control']= data['description_word'].apply(lambda x: 
                                                1 if ('климат-контроль') in x else 0)
data['multi_rudder']= data['description_word'].apply(lambda x: 
                                                1 if ('мультифункциональный' and 'руль') in x else 0)
data['power_steering']= data['description_word'].apply(lambda x: 
                                                1 if ('гидроусилитель' or 'гидро' or 'усилитель' and 'руля') in x else 0)
data['light_and_rain_sensors']= data['description_word'].apply(lambda x: 
                                                1 if ('датчики' and 'света' and 'дождя') in x else 0)
data['сarbon_body_kits']= data['description_word'].apply(lambda x: 
                                                1 if ('карбоновые' and 'обвесы') in x else 0)
data['rear_diffuser_rkp']= data['description_word'].apply(lambda x: 
                                                1 if ('задний' and 'диффузор') in x else 0)
data['door_closers']= data['description_word'].apply(lambda x: 
                                                1 if ('доводчики' and 'дверей') in x else 0)
data['rear_view_camera']= data['description_word'].apply(lambda x: 
                                                1 if ('камера' or 'видеокамера' and 'заднего' and 'вида') in x else 0)
data['amg']= data['description_word'].apply(lambda x: 
                                                1 if ('amg') in x else 0)
data['bi_xenon_headlights']= data['description_word'].apply(lambda x: 
                                                1 if ('биксеноновые' and 'фары') in x else 0)
data['from_salon']= data['description_word'].apply(lambda x: 
                                                1 if ('рольф' or 'панавто' or 'дилер' or 'кредит' or 'ликвидация') in x else 0)
data['alloy_wheels']= data['description_word'].apply(lambda x: 
                                                1 if ('легкосплавные' or 'колесные' or 'диски') in x else 0)
data['parking_sensors']= data['description_word'].apply(lambda x: 
                                                1 if ('парктроник' or 'парктронник') in x else 0)
data['dents']= data['description_word'].apply(lambda x: 
                                                1 if ('вмятины' or 'вмятина' or 'царапина' or 'царапины' or 'трещина') in x else 0)
data['roof_with_panoramic_view']= data['description_word'].apply(lambda x: 
                                                1 if ('панорамная' and 'крыша') in x else 0)

In [19]:
spis_eng = [str(i) for i in data['engineDisplacement'].unique()]

In [20]:
def engineDisplacement(row):
    row = str(row)
    engine = re.findall('\d\.\d', row)
    if engine == []:
        return None
    return float(engine[0])

In [21]:
data['engineDisplacement'] = data['name'].apply(engineDisplacement)

In [22]:
data['Владение_is_none'] = pd.isna(data['Владение']).astype('uint8')
data['Владельцы'] = data['Владельцы'].fillna(3.0)
data['engineDisplacement'] = data['engineDisplacement'].fillna(2.0) 

In [23]:
def num_of_months(row):
    if pd.notnull(row):
        list_ownership = row.split()
        if len(list_ownership) == 2:
            if list_ownership[1] in ['год', 'года', 'лет']:
                return int(list_ownership[0])*12
            return int(list_ownership[0])
        return int(list_ownership[0])*12 + int(list_ownership[3])

In [24]:
data['month_ownership'] = data['Владение'].apply(num_of_months)

In [25]:
data['month_ownership'].fillna(data['month_ownership'].median(),inplace=True)

Удаляем лишние колонки и обработанными данными. И проверяем лесть ли пропуски в данных.

In [26]:
data.drop(['description', 'Комплектация', 'name', 'description_word', 'Владение'], axis = 1, inplace = True)
print('Количество пропусков: ', data.isna().sum().sum())

Количество пропусков:  0


Преобразуем в дамми параметры.

In [27]:
data = pd.get_dummies(data, columns = ['bodyType', 'brand', 'color', 'fuelType', 'vehicleConfiguration',
                                           'Привод', 'Руль', 'Владельцы', 'ПТС', 'engineDisplacement'], dummy_na=True)

In [29]:
# отделяем трейновые и тестовые данные
train_data = data.query('sample==1').drop('sample', axis = 1)
test_data = data.query('sample==0').drop(['sample', 'Price'], axis = 1).reset_index(drop=True)

In [30]:
X = train_data.drop(['Price'], axis=1).reset_index(drop=True)
y = train_data['Price'].values # target           

In [31]:
models = [RandomForestRegressor(n_estimators =250,random_state = RANDOM_SEED, n_jobs = -1, verbose = 1),
         BaggingRegressor(ExtraTreeRegressor(random_state=RANDOM_SEED), random_state=RANDOM_SEED)]

def stacking_model_predict(models, X, y, test_data, sample_submission):
    for model_ in tqdm(models):
        model_.fit(X, y)
        pred_subm = model_.predict(test_data)
        sample_submission[str(model_)[:6]] = pred_subm
        sample_submission[str(model_)[:6]] = sample_submission[str(model_)[:6]].apply(lambda x: np.exp(x) )
        sample_submission[str(model_)[:6]] = sample_submission[str(model_)[:6]].apply(lambda x: round(x/1000)*1000)
    sample_submission['price'] = sample_submission.iloc[:,2:].mean(axis=1)
    sample_submission[['id', 'price']].to_csv(f'submission_v{VERSION}.csv', index=False)
    sample_submission.head(10)

stacking_model_predict(models, X, y, test_data, sample_submission)

  0%|          | 0/2 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  2.0min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.0s finished
100%|██████████| 2/2 [02:17<00:00, 68.66s/it] 
